In [1]:
using MPSGE_MP

using JLD2, CSV

using DataFrames

using JuMP,PATHSolver

using JuMP.Containers

import GamsStructure, GDX

In [2]:
gams_sols = CSV.read("../state_sols_gams.csv",DataFrame; stringtype=String) |>
    x -> rename(x, :level => :gams_value);

In [3]:
## Julia generated data
 GU = GamsStructure.load_universe("../WiNDCdatabase",raw_text=false);

#GU = load_universe_gdx(
#    raw"C:\Users\mphillipson\Documents\WiNDC\windc_build\core\WiNDCdatabase.gdx",
#    aliases = Dict(:s => [:g])
#);

Sets

m => Margins (trade or transport)
s => BEA Goods and sectors categories => Aliases: [:g]
g => BEA Goods and sectors categories => Aliases: [:s]
gm => Commodities employed in margin supply
yr => Years in WiNDC Database
r => Regions in WiNDC Database

Parameters

bopdef0_ => (:yr, :r) => Balance of payments (closure parameter)
dm0_ => (:yr, :r, :g, :m) => Margin supply from the local market
nd0_ => (:yr, :r, :g) => Regional demand from national marke
id0_ => (:yr, :r, :g, :s) => Regional intermediate demand
a0_ => (:yr, :r, :g) => Domestic absorption
ld0_ => (:yr, :r, :s) => Labor demand
rx0_ => (:yr, :r, :g) => Re-exports
c0_ => (:yr, :r) => Total final household consumption
s0_ => (:yr, :r, :g) => Total supply
xn0_ => (:yr, :r, :g) => Regional supply to national market
md0_ => (:yr, :r, :m, :g) => Margin demand
ys0_ => (:yr, :r, :s, :g) => Regional sectoral output
xd0_ => (:yr, :r, :g) => Regional supply to local market
ty0_ => (:yr, :r, :s) => Production tax rate
hhadj0_ => (:yr

In [4]:
GamsStructure.@extract_sets_as_vector(GU,    
    # Sets
    m => M,
    s => S,
    g => G,
    gm => GM,
    yr => YR,
    r => R,
)

GamsStructure.@extract(GU, 
    # Parameters
    bopdef0_ => bopdef0,
    dm0_ => dm0,
    nd0_ => nd0,
    id0_ => id0,
    a0_ => a0,
    ld0_ => ld0,
    rx0_ => rx0,
    c0_ => c0,
    s0_ => s0,
    xn0_ => xn0,
    md0_ => md0,
    ys0_ => ys0,
    xd0_ => xd0,
    ty0_ => ty0,
    hhadj0_ => hhadj0,
    cd0_ => cd0,
    fe0_ => fe0,
    m0_ => m0,
    dd0_ => dd0,
    i0_ => i0,
    tm0_ => tm0,
    gdp0_ => gdp0,
    nm0_ => nm0,
    kd0_ => kd0,
    g0_ => g0,
    x0_ => x0,
    ta0_ => ta0,
    yh0_ => yh0
);


#This is the slow part

#ty = ty0 #deepcopy(ty0)
#ta = ta0 #deepcopy(ta0)
#tm = deepcopy(tm0)

#tm[:yr,:r,:g] = 0*tm[:yr,:r,:g]
1;

In [7]:
yr = Symbol(2017)

state = MPSGEModel()


@parameters(state, begin
    ta, ta0, (index = [YR,R,G],)
    ty, ty0, (index = [YR,R,S],)
    tm, tm0, (index = [YR,R,G],)
end)


@sectors(state,begin
    Y, (index = [R,S],  description = "Production")
    X, (index = [R,G],  description = "Disposition")
    A, (index = [R,G],  description = "Absorption")
    C, (index = [R],    description = "Aggregate final demand")
    MS,(index = [R,M],  description = "Margin supply")
end)


@commodities(state,begin
    PA, (index = [R,G], description = "Regional market (input)")
    PY, (index = [R,G], description = "Regional market (output)")
    PD, (index = [R,G], description = "Local market price")
    PN, (index = [G],   description = "National market")
    PL, (index = [R],   description = "Wage rate")
    PK, (index = [R,S], description = "Rental rate of capital")
    PM, (index = [R,M], description = "Margin price")
    PC, (index = [R],   description = "Consumer price index")
    PFX, (              description = "Foreign exchange",)
end)

@consumer(state, RA, index = [R], description = "Representative agent")




for r∈R,s∈S
    @production(state, Y[r,s], [t=0, s=0, va=>s=1], begin
        [@Output(PY[r,g],ys0[yr,r,s,g], t, taxes = [Tax(RA[r],ty[yr,r,s])], reference_price = 1-ty0[yr,r,s]) for g∈G]...
        [@Input(PA[r,g], id0[yr,r,g,s], s) for g∈G]...
        @Input(PL[r],    ld0[yr,r,s],   va)
        @Input(PK[r,s],  kd0[yr,r,s],   va)
    end)
end


for r∈R,g∈G
    @production(state, X[r,g], [t = 4, s = 0], begin
        @Output(PFX,     x0[yr,r,g] - rx0[yr,r,g],  t)
        @Output(PN[g],   xn0[yr,r,g],               t)
        @Output(PD[r,g], xd0[yr,r,g],               t)
        @Input(PY[r,g],  s0[yr,r,g],                s)
    end)
end



for r∈R,g∈G
    @production(state, A[r,g], [t = 0, s = 0, d=>s = 2, dm => d = 4], begin
        @Output(PA[r,g], a0[yr,r,g],    t, taxes=[Tax(RA[r],ta[yr,r,g])], reference_price = 1-ta0[yr,r,g])
        @Output(PFX,     rx0[yr,r,g],   t)
        [@Input(PM[r,m], md0[yr,r,m,g], s) for m∈M]...
        @Input(PFX,      m0[yr,r,g],    d, taxes = [Tax(RA[r],tm[yr,r,g])], reference_price=1+tm0[yr,r,g])
        @Input(PN[g],    nd0[yr,r,g],   dm)
        @Input(PD[r,g],  dd0[yr,r,g],   dm)
    end)
end


for r∈R,m∈M
    @production(state, MS[r,m], [t=0, s = 0], begin
        @Output(PM[r,m],  sum(md0[yr,r,m,gm] for gm∈GM), t)
        [@Input(PN[gm],   nm0[yr,r,gm,m],                s) for gm∈GM]...
        [@Input(PD[r,gm], dm0[yr,r,gm,m],                s) for gm∈GM]...
    end)
end


for r∈R
    @production(state, C[r], [ t = 0, s = 1], begin
            @Output(PC[r],   c0[yr,r],    t)
            [@Input(PA[r,g], cd0[yr,r,g], s) for g∈G]...
    end)
end


for r∈R
    add_demand!(state, RA[r],
        [ScalarDem(PC[r], c0[yr,r])],
        [
            [ScalarEndowment(PY[r,g], yh0[yr,r,g]) for g∈G];
            [ScalarEndowment(PFX, bopdef0[yr,r] + hhadj0[yr,r])];
            [ScalarEndowment(PA[r,g], -g0[yr,r,g] - i0[yr,r,g]) for g∈G];
            [ScalarEndowment(PL[r], sum(ld0[yr,r,s] for s∈S))];
            [ScalarEndowment(PK[r,s], kd0[yr,r,s]) for s∈S]
        ]
    )
end


#jm = build!(state);

In [8]:
build!(state)

# Benchmark

In [9]:
fix(RA[:CA], c0[yr,:CA])

solve!(state; cumulative_iteration_limit = 0)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_9E61.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 25826

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 5.1403e-09           I 0.0e+00 1.8e-09 (market_clearanc)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.969000
Residual. . . . . . . . 5.140277e-09
Postsolved residual: 5.1403e-09


# Counterfactual

In [10]:
set_value!(tm,0)

solve!(state)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:   428 Single:     0 Double:     0
Preprocessed size   : 25398

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.3510e+01             0.0e+00 (market_clearance[PN(:mo)
    1     1     0 25398 4.4614e-02  1.0e+00    0.0e+00 (market_clearance[PFX)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 4.4614e-02           I 0.0e+00 3.3e-02 (market_clearanc)
    1     1     3     3 2.4721e-04  1.0e+00 SO 0.0e+00 1.8e-04 (market_clearanc)
    2     1     4     4 4.6634e-08  1.0e+00 SO 0.0e+00 2.8e-08 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 4.3740

In [11]:
df = generate_report(state) |>
    x -> transform(x,
        :var => ByRow(JuMP.name) => :variable
    ) |>
    x -> outerjoin(x, gams_sols, on = :variable) |>
    x -> dropmissing(x);

In [12]:
df |>
    x -> transform(x,
        [:value,:gams_value] => ((a,b) -> abs.(a-b)) => :diff
    ) |>
    x -> sort(x, :diff, rev = true)

Row,var,value,margin,variable,gams_value,diff
,GenericV…,Float64,Float64,String,Float64,Float64
1,RA[TX],1062.6,-2.27374e-13,RA[TX],1062.9,0.305543
2,RA[FL],902.007,-1.13687e-13,RA[FL],902.286,0.278943
3,RA[NY],913.913,-1.13687e-13,RA[NY],913.702,0.211367
4,RA[MI],384.83,-1.7053e-13,RA[MI],385.04,0.2109
5,RA[PA],541.287,3.39924e-11,RA[PA],541.436,0.149573
6,RA[NJ],413.06,2.27374e-13,RA[NJ],413.167,0.106283
7,RA[OH],433.433,-1.13687e-13,RA[OH],433.536,0.103868
8,RA[SC],178.056,1.98952e-13,RA[SC],178.14,0.0835043
9,RA[MO],228.354,2.55795e-13,RA[MO],228.435,0.0814896


In [13]:
compensated_demand(Y[:WI,:ppd], PY[:WI,:ppd])

-13.418081211404933

In [14]:
production(Y[:WI,:ppd])

$Production: Y(:WI, :ppd)
:t = 0
  O:PY(:WI, :fmt)    Q: 7.477409663278371e-5    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :nmp)    Q: 0.004582736338749625    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :tex)    Q: 0.02491516529223812    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :con)    Q: 0.005986539496328994    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :pla)    Q: 0.05201752574265005    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :ppd)    Q: 13.418081211404933    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :che)    Q: 0.009845297912877846    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :tsv)    Q: 0.043859396899654964    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :mmf)    Q: 0.004507176972946336    P:0.9905916270983147    A:RA(:WI,)    T:ty[2017,WI,ppd]
  O:PY(:WI, :cep)    Q: 0.043949220

# Testing 

In [ ]:
extract_constraint(jm,constraint_name,ind) = constraint_object(jm[constraint_name][ind]).func[1]

In [ ]:
df |>
    x -> subset(x,
        :variable => ByRow(==("A[WI,ppd]"))
    )

In [ ]:
P = production(A[:WI,:ppd])

In [ ]:
extract_constraint(jm,:zero_profit,A[:WI,:ppd])

In [ ]:
tmp = collect(keys(P.nested_compensated_demand))[end][1]

In [ ]:
MPSGE_MP.quantity(tmp)

In [ ]:
compensated_demand(A[:WI,:ppd],PFX)

In [ ]:
P

In [ ]:
rx0[yr,:WI,:ppd]

In [ ]:
print(tau(A[:WI,:ppd],RA[:WI]))

# Fix to GAMS solutions and resolve

Commented out for run all purposes

In [ ]:
for row in eachrow(df)
    fix(row[:var], row[:gams_value], force=true)
end

In [ ]:
solve!(state; :cumulative_iteration_limit => 0)

In [ ]:
df_ = generate_report(state) |>
    x -> transform(x,
        :margin => ByRow(abs) => :abs_margin
    ) |>
    x -> sort(x, :abs_margin, rev=true)

In [ ]:
df_ |>
    x -> subset(x,
        :abs_margin => ByRow(>(1e-3))
    )|>
    x -> transform(x,
        :var => ByRow(JuMP.name) => :variable
    ) |>
    x -> transform(x,
        :variable => (y -> replace.(y, r"^(\w*)\[.*" => s"\1")) => :base_name
    )

In [ ]:
sectors(PD[:FL,:hou])

In [ ]:
sec = A[:FL,:hou]
com = PD[:FL,:hou]

production(sec)

In [ ]:
compensated_demand(sec,com)

In [ ]:
eqn = extract_constraint(jm, :market_clearance, com)

print(eqn)

In [ ]:
value(eqn)

In [ ]:
value(extract_constraint(jm,:zero_profit, sec))

In [ ]:
value(extract_constraint(jm,:zero_profit, A[:FL,:hou]))

In [ ]:
nd0[yr,:FL,:hou]/(nd0[yr,:FL,:hou] + dd0[yr,:FL,:hou])

In [ ]:
value(A[:FL,:hou])

In [ ]:
value(compensated_demand(A[:FL,:hou],PD[:FL,:hou]))

In [ ]:
value(compensated_demand(X[:FL,:hou],PD[:FL,:hou]))

In [ ]:
xn0[yr,:FL,:hou]

In [ ]:
value(X[:FL,:hou])

In [ ]:
value(compensated_demand(X[:FL,:hou], PD[:FL,:hou]))*value(X[:FL,:hou])/value(A[:FL,:hou])

In [ ]:
value(compensated_demand(A[:FL,:hou], PD[:FL,:hou])) -80.99087736139923

In [ ]:
compensated_demand(A[:FL,:hou], PD[:FL,:hou])

In [ ]:
m0[yr,:FL,:hou]

In [ ]:
diff = 0.017839384010329695

1/(value(PD[:FL,:hou]) * sqrt(diff/dd0[yr,:FL,:hou]))